Segment and Cluster Toronto Neighborhoods

Table of Contents:
-Project Overview
-Data Overview
-Data Wrangling
-Exploratory Data Analysis
-Results & Discussion
-Conclusion

Assignment Overview:
The purpose of this project is to leverage Foursquare's API to explore neighborhoods in Toronto, CA in order to segment and cluster them by venue categories. This is a great practice to align residential neighborhoods/boroughs to personal interest's and hobbies. The 'explore' function will be used to get the most common venue categories in each neighborhood. K-means clustering algorithm will be implemented to cluster the final results. Finally, pythons Folium library will be used to visualize the neighborhoods in Toronto and their emerging clusters.

Data Overview:
Using data provided by Foursquare, venues and venue categories information will be assessed to determine distinguishing features among boroughs.

Data sources will include the web (web scraping), a .csv file specifying geospatial data of the boroughs in Toronto, and data explored utilizing Foursquare's API. Postal code data will be scraped from 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'. A final DataFrame combining all the desired data will be constructed prior to cluster analysis.

The cluster analysis k-value will be optimized based on the inital results at k = 5. K-values from 3-10 will ultimately be analyzed.

Installing / Importing Libraries: 

In [1]:
conda update numpy

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - numpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_1         157 KB
    openssl-1.1.1f             |       h7b6447c_0         3.8 MB
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following packages will be UPDATED:

    certifi: 2019.11.28-py36h9f0ad1d_1 conda-forge --> 2019.11.28-py36_1
    openssl: 1.1.1f-h516909a_0         conda-forge --> 1.1.1f-h7b6447c_0


certifi-2019.11.28   | 157 KB    | ##################################### | 10

In [2]:
conda upgrade --all

Solving environment: | 
  - conda-forge::libgfortran-3.0.0-1
  - defaults::libgfortran-3.0.0done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxml2-2.9.9              |       hea5a465_1         2.0 MB
    pysocks-1.7.1              |           py36_0          30 KB
    pytorch-1.4.0              |cpu_py36h7e40bad_0        33.8 MB
    freeglut-3.0.0             |       hf484d3e_5         251 KB
    ninja-1.9.0                |   py36hfd86e86_0         1.6 MB
    pthread-stubs-0.3          |       h0ce48e5_1           5 KB
    kiwisolver-1.1.0           |   py36he6710b0_0          90 KB
    readline-8.0               |       

In [3]:
pip install geocoder

     |████████████████████████████████| 98 kB 4.9 MB/s eta 0:00:011
     |████████████████████████████████| 82 kB 1.7 MB/s eta 0:00:011
     |████████████████████████████████| 829 kB 36.5 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=3bbd0f4323c9ab7534a976ae77b6c146759542890b2850ba60c009f5ed1ef7c2
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/6e/9c/ed/4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import required libraries and packages
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from PIL import Image
import requests

#!conda install -c conda-forge geocoder -y
import geocoder

# Matplotlib and associated plotting modules
%matplotlib inline
import matplotlib.cm as cm

import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
%pip install lxml
print('lxml installed')

Note: you may need to restart the kernel to use updated packages.
lxml installed


Data Wrangling:
Below, pandas 'read_html' command is used to import the raw data from the wikipedia link provided. This, however reads all data on the page, so a table to pull data from i.e. "canada[0]" must be specified. This reads the first table in the page.

In [6]:
# Creating Dataframe from Wiki Table
path = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'    
canada = pd.read_html(path, header=0)
cn_table = canada[0]
cn_table.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [7]:
# Store original DataFrame
cn_table_original = cn_table.copy()

In [9]:
# Data Wrangling - Filter Borough's and Neighbhorhoods
cn_table = cn_table.query('Borough != "Not assigned"')
cn_table.loc[:,'Neighborhood'] = np.where(cn_table['Neighborhood'] == 'Not assigned', cn_table['Borough'], cn_table['Neighborhood'])
cn_table.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Below, the table is grouped by Postcode and an aggregate function is used to combine the remaining columns with a ',' if the Postcode's are identical. For this analysis, # if eighborhoods in order to analyze venues/living in each neighborhood and not each postal code.

In [10]:
# drop cells with a borough that is Not assigned
cn_table = cn_table[cn_table.Borough != 'Not assigned'].reset_index(drop=True)
cn_table.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [11]:
cn_table = cn_table.groupby('Postal code', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
cn_table.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
# group neighborhoods in the same borough
cn_table = cn_table.groupby(["Postal code", "Borough"],as_index=False).agg(lambda x: ", ".join(x))
cn_table.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in cn_table.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
        
cn_table.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
cn_table.head(20)

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


In [15]:
cn_table.shape

(103, 3)

In [16]:
# Longitude and latitude data from .csv file is read. Column 'Postal Code' is renamed to 'Postcode' so it can be merged with original dataframe.
lat_long_path = 'http://cocl.us/Geospatial_data'
latlongdf= pd.read_csv(lat_long_path)
latlongdf.rename(columns={'Postal Code':'Postalcode'},inplace=True)

In [17]:
cn_table.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
# Longitude and latitude data from .csv file is read. Column 'Postal Code' is renamed to 'Postcode' so it can be merged with original dataframe.
lat_long_path = 'http://cocl.us/Geospatial_data'
latlongdf= pd.read_csv(lat_long_path)
latlongdf.rename(columns={'Postal Code':'Postal code'},inplace=True)

In [22]:
df = pd.merge(cn_table,latlongdf, on='Postal code')
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [23]:
# For analysis, boroughs containing the string 'Toronto' are filtered.
df = df[df['Borough'].str.contains("Toronto")]
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
42,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [24]:
# Use Nominatim to get geo data for Toronto, CA to use in Folium library.
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [25]:
!pip install folium
print('folium installed and imported')

folium installed and imported


In [28]:
# Use Folium to render map of Toronto, CA
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11.2, width = '50%', height = '50%')

for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, 
                        parse_html=False).add_to(toronto_map)  
    
toronto_map;

In [ ]:
from IPython.display import Image
Image(filename='toronto1.png')

FourSquare API : 

In [29]:
# Foursqure Access Credentials
# CLIENT_ID = 'INSERT CLIENT ID'
# CLIENT_SECRET = 'INSERT SECRET'
CLIENT_ID = 'TJZPZ1S54SRN0K4YRUZR3JHSBBR05JH4B4V4KWKLBKIQDS04' # your Foursquare ID
CLIENT_SECRET = 'TZN5E2IAWQR1TYI10D2DS2KYGS31RVEGMQXXISYFPDIODM1S' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: TJZPZ1S54SRN0K4YRUZR3JHSBBR05JH4B4V4KWKLBKIQDS04
CLIENT_SECRET:TZN5E2IAWQR1TYI10D2DS2KYGS31RVEGMQXXISYFPDIODM1S


In [30]:

LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_toronto=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_toronto.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_toronto in venues_toronto for item in venue_toronto])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 
                  'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [31]:
# Pull top venues for each Neighbourhood in Toronto
toronto_venues = getNearbyVenues(names = df['Neighborhood'], latitudes = df['Latitude'], longitudes = df['Longitude'])

Exploratory Data Analysis:

In [32]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Assign dummy variables to each neighborhood and venues in that neighborhood in order to take the mean of the data and analyze the frequency of each venue category in a neighborhood.

In [33]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.018182,0.0,0.0,0.0,0.0000,0.018182,0.0,0.018182,0.036364,0.0,0.000000,0.0,0.018182,0.018182,0.0,0.036364,0.0,0.0,0.018182,0.0,0.000000,0.000000,0.000000,0.0,0.018182,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.018182,0.036364,0.0,0.0,0.0,0.0,0.036364,0.000000,0.0,0.0,0.000000,0.0,0.054545,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.018182,0.000000,0.018182,0.000000,0.018182,0.0,0.018182,0.0,0.0,0.0,0.0,0.018182,0.000000,0.018182,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.036364,0.0000,0.0,0.0,0.0

In [34]:
# Top 10 venue categories for each neighborhood
num_top_venues = 10

for Neighborhood in toronto_grouped['Neighborhood']:
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == Neighborhood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

In [35]:
# Sort above data of venues in each neighborhood by most popular.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
# Create dataframe of top 10 most common venues in each neighborhoods. 
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Farmers Market,Greek Restaurant
1,Brockton / Parkdale Village / Exhibition Place,Café,Breakfast Spot,Coffee Shop,Grocery Store,Gym,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant,Intersection
2,Business reply mail Processing CentrE,Yoga Studio,Auto Workshop,Skate Park,Brewery,Smoke Shop,Spa,Farmers Market,Fast Food Restaurant,Restaurant,Burrito Place
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Sculpture Garden,Rental Car Location,Plane,Boutique,Harbor / Marina,Airport Gate
4,Central Bay Street,Coffee Shop,Italian Restaurant,Burger Joint,Japanese Restaurant,Café,Sandwich Place,Thai Restaurant,Gym / Fitness Center,Bubble Tea Shop,Ice Cream Shop


Clustering:

In [37]:
# Initialize 5 k clusters and use KMeans to assign labels to each Neighborhood section.
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis = 1)
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [38]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()


,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Park,Trail,Pub,Health Food Store,Dumpling Restaurant,Donut Shop,Eastern European Restaurant,Doner Restaurant,Dance Studio,Dog Run
41,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Grocery Store,Pub,Pizza Place,Lounge
42,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,2,Park,Sushi Restaurant,Coffee Shop,Movie Theater,Pet Store,Pub,Burrito Place,Restaurant,Liquor Store,Brewery
43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place,Cheese Shop
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [39]:
toronto_merged = toronto_merged.dropna(how='any',axis=0) 
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

Cluster Mapping using Folium

In [40]:
# Plot cluster map of Tornoto
map_clusters = folium.Map(location=[latitude, longitude], zoom_start = 11.2, width = '50%', height = '50%')

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color = rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1],
                        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters;

In [ ]:
Image(filename='toronto2.png')

Cluster Analysis
Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

Cluster 1

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Park,Trail,Pub,Health Food Store,Dumpling Restaurant,Donut Shop,Eastern European Restaurant,Doner Restaurant,Dance Studio,Dog Run
44,Central Toronto,0,Park,Swim School,Bus Line,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
50,Downtown Toronto,0,Park,Playground,Trail,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Cluster 2

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Grocery Store,Pub,Pizza Place,Lounge
42,East Toronto,2,Park,Sushi Restaurant,Coffee Shop,Movie Theater,Pet Store,Pub,Burrito Place,Restaurant,Liquor Store,Brewery
43,East Toronto,2,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place,Cheese Shop
45,Central Toronto,2,Sandwich Place,Convenience Store,Food & Drink Shop,Asian Restaurant,Hotel,Department Store,Breakfast Spot,Park,Gym,Dumpling Restaurant
46,Central Toronto,2,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Chinese Restaurant,Café


Cluster 3

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Grocery Store,Pub,Pizza Place,Lounge
42,East Toronto,2,Park,Sushi Restaurant,Coffee Shop,Movie Theater,Pet Store,Pub,Burrito Place,Restaurant,Liquor Store,Brewery
43,East Toronto,2,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place,Cheese Shop
45,Central Toronto,2,Sandwich Place,Convenience Store,Food & Drink Shop,Asian Restaurant,Hotel,Department Store,Breakfast Spot,Park,Gym,Dumpling Restaurant
46,Central Toronto,2,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Chinese Restaurant,Café
